<a href="https://colab.research.google.com/github/frantropy/multi-eGO/blob/main/Multi_eGO_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup **multi-_e_GO**
#@markdown NEEDS TO BE RUN TWICE AND FIRST TIME GIVES ERROR
from google.colab import drive
import os
import shutil
#from google.colab import drive
#drive.mount('/content/drive')
MEGO_ROOT = '/content/multi-eGO'

!git clone https://github.com/multi-ego/multi-eGO.git
!pip install -q condacolab
import condacolab
condacolab.install()
!pip install -U pandas numpy parmed ipywidgets==7.6.5
condacolab.check()

In [ ]:
#@title Prepare inputs
for d in os.listdir(f'{MEGO_ROOT}/inputs'):
  if '.zip' in d:
    out = d.replace('.zip', '')
    !unzip {MEGO_ROOT}/inputs/{d} -d {MEGO_ROOT}/inputs/
for d in os.listdir(f'{MEGO_ROOT}/test/test_inputs'):
  out_path = MEGO_ROOT + '/inputs/' + d.split('/')[-1]
  if not os.path.isdir(out_path) and 'ref' in d:
    shutil.copytree(f'{MEGO_ROOT}/test/test_inputs' + '/' + d, out_path)

In [17]:
#@title Select **multi-_e_GO** system or DIY
import ipywidgets as widgets
import os
import pandas as pd
import numpy as np

class MegoSetupView:
  def __init__(self, MEGO_ROOT):
    self.root = MEGO_ROOT
    self.systems_path = f'{MEGO_ROOT}/inputs'
    self.systems_list = [ x for x in os.listdir(self.systems_path) if os.path.isdir(f'{self.systems_path}/{x}') ]
    self.system_parameters = {
        'gpref': {
            'epsilon': 0.235,
            'train': 'md_ensemble',
            'reference': 'reference',
            'symmetry': ''
        },
        'abetaref': {
            'epsilon': 0.33,
            'train': 'native_MD',
            'reference': 'ref',
            'symmetry': f'{self.systems_path}/abetaref/aa_sym',
        },
        'ttrref': {
            'epsilon': 0.33,
            'train': 'native_MD',
            'reference': 'reference',
            'symmetry': f'{self.systems_path}/ttrref/aa_sym'
        }
    }
    if self.systems_list == ['.gitignore']:
      print(f"Systems empty! Please make sure the path {self.systems_path} exists!")
      raise KeyboardInterrupt
    self.systems = widgets.Dropdown(
      options=self.systems_list,
      description="System:"
    )

    self.epsilon = widgets.FloatText(
      value=0.3,
      min=0.1,
      max=5.0,
      step=0.01,
      description='epsilon:',
      disabled=True
    )

    self.egos = widgets.Dropdown(
      options=['rc', 'production'],
      description='Egos:',
      disabled=False
    )

    self.train_list = [ d for d in os.listdir(f'{self.root}/inputs/{self.systems.value}') if os.path.isdir(d) ]
    self.train = widgets.Dropdown(
      options=self.train_list,
      description="Train:",
      disabled=True
    )

    self.reference_list = [ d for d in os.listdir(f'{self.root}/inputs/{self.systems.value}') if os.path.isdir(d) ]
    self.reference = widgets.Dropdown(
        options=self.reference_list,
        description="Reference:",
        disabled=False,
    )

    self.symmetry = widgets.Text(
      value='',
      placeholder=f'symmetry_file',
      description='Symmetry:',
      disabled=True
    )

    ######## EVENT HANDLERS #############
    self.systems.observe(self.on_change_system, names='value')
    self.egos.observe(self.on_change_egos, names='value')

    self.run = button = widgets.Button(
      description='Run',
      disabled=False,
      button_style='', # 'success', 'info', 'warning', 'danger' or ''
      tooltip='Click me',
      # icon='play'
    )
    self.run.on_click(self.run_mego)

    self.out = widgets.Output(layout={'border': '1px solid black'})

    # put all components up for display
    display(self.systems)
    display(self.egos)
    display(self.reference)
    display(self.epsilon)
    display(self.train)
    display(self.symmetry)
    display(self.run)
    display(self.out)

    self.update_train(self.systems.value)
    self.update_reference(self.systems.value)

  def run_mego(self, click):
    ##### CHECK ALL VALUES #####
    if float(self.epsilon.value) > 1.0 or float(self.epsilon.value) < 0.1:
      self.out.clear_output()
      with self.out:
        print('ERROR: please set an epsilon value between 0.1 and 1.0.')
    if os.path.isfile(self.symmetry.value):
      self.out.clear_output()
      with self.out:
        print('ERROR: please give a valid symmetry file path.')

    exec_string = f"python {self.root}/multiego.py"
    exec_string += f" --system {self.systems.value}"
    exec_string += f" --egos {self.egos.value}"
    exec_string += f" --reference {self.reference.value}"
    if self.egos.value == 'production':
      exec_string += f" --epsilon {self.epsilon.value}"
      exec_string += f" --train {self.train.value}"
      if self.symmetry.value:
        exec_string += f" --symmetry {self.symmetry.value}"
    self.out.clear_output()
    with self.out:
      print('Running multi-eGO...')
    exit_code = os.system(exec_string)
    if exit_code == 0:
      self.out.clear_output()
      with self.out:
        out_path = f'{self.root}/outputs/{self.systems.value}_{self.egos.value}'
        if self.egos.value == 'production':
          out_path += f'_e{self.epsilon.value}_{self.epsilon.value}'
        print(f'Done! Outputs can be found in {out_path}')
    else:
      with self.out:
        print("ERROR: something went wrong!")
        print(f"'{exec_string}' did not work")

  def update_train(self, new_sys):
    self.train_list = [ d for d in os.listdir(f'{self.root}/inputs/{self.systems.value}') ]
    self.train_list = [ d for d in self.train_list if os.path.isdir(f'{self.root}/inputs/{self.systems.value}/{d}') and not d.startswith('.') ]
    self.train.options = self.train_list

  def update_reference(self, new_sys):
    self.reference_list = [ d for d in os.listdir(f'{self.root}/inputs/{self.systems.value}') ]
    self.reference_list = [ d for d in self.reference_list if os.path.isdir(f'{self.root}/inputs/{self.systems.value}/{d}') and not d.startswith('.') ]
    self.reference.options = self.reference_list

  def on_change_system(self, change):
    new_sys = change['new']
    self.update_train(new_sys)
    self.update_reference(new_sys)
    if new_sys in self.system_parameters.keys():
      self.reference.value = self.system_parameters[new_sys]['reference']
      self.epsilon.value = self.system_parameters[new_sys]['epsilon']
      self.train.value = self.system_parameters[new_sys]['train']
      self.symmetry.value = self.system_parameters[new_sys]['symmetry']

  def on_change_egos(self, change):
    disabled = change['new'] == 'rc'
    self.epsilon.disabled = disabled
    self.train.disabled = disabled
    self.reference.disabled = disabled
    self.symmetry.disabled = disabled

app = MegoSetupView(MEGO_ROOT)

Dropdown(description='System:', options=('lyso-bnz_ref', 'ttrref', 'abetaref', 'testino', '.ipynb_checkpoints'…

Dropdown(description='Egos:', options=('rc', 'production'), value='rc')

Dropdown(description='Reference:', options=(), value=None)

FloatText(value=0.3, description='epsilon:', disabled=True, step=0.01)

Dropdown(description='Train:', disabled=True, options=(), value=None)

Text(value='', description='Symmetry:', disabled=True, placeholder='symmetry_file')

Button(description='Run', style=ButtonStyle(), tooltip='Click me')

Output(layout=Layout(border='1px solid black'))